In [5]:
import os
from Bio import Entrez
import json

In [6]:
vazy2_path:str = "./../../../../testymolo/media/data/"
vazy2_list:list = [ 'Modulo', 'Profile', 'Organism', 'PolyProtein', 'Protein', 'Subseq', ]
vazy2:dict= {}

def Load(param:str) -> list:
    fileName:str = os.path.join(vazy2_path, param+'.temp.json')
    with open(fileName) as handle:
        return json.load(handle)
for tabName in vazy2_list:
    vazy2[tabName] = Load(tabName)

### trace

In [6]:
trace:dict = {}
for profile in vazy2['Profile']:
    k = profile['id']
    modId = profile['modulo']
    if modId.startswith('F'):
        trace[modId] =  []
        #filter
        for subSeq in vazy2['Subseq']:
            if subSeq['profile'] == k:
                trace[modId].append((subSeq['id'], subSeq['origin'], subSeq['profile'], subSeq['start'], subSeq['end']))
trace
with open("./vazi2_mod_trace.json", 'w') as out :
    out.write(json.dumps(trace))

# Extract profile fasta

In [7]:
for mod, SubSeqs in trace.items():
    fasta:list = []
    for ss in SubSeqs:
        #filter
        for item in vazy2['Protein']:
            if item['id'] == ss[1]:
                header:str = '>'+item['header']+f":[{ss[-2]}-{ss[-1]}]\n"
                sequence:str = item['sequence'][ss[-2]-1:ss[-1]-1]+'\n'
                fasta.append(header+sequence)
                break
    #endfor
    with open(f"./fasta/{mod}.fasta", 'w') as out:
        out.write(''.join(fasta))
        

### Alignment fasta

In [21]:
from Bio.Align.Applications import MuscleCommandline
import subprocess 

for file in os.listdir("./fasta"):
    #print(file)
    inFile:str = os.path.join("./fasta", file)
    out:str = os.path.join('./aln', file.split(".fasta")[0]+'.aln'+'.fasta')
    
    command:str = f"muscle -in {inFile} -out {out} -quiet"
    print(command)  # adding "-quiet" option ... doen't work in v3.8 ?! (but does in v5.)
    subprocess.Popen(command.split(' '), stdout=subprocess.DEVNULL)  # command must be a list of words



muscle -in ./fasta/F191.fasta -out ./aln/F191.aln.fasta -quiet


### transfer  

* *foreach PP -> Map les nsp  =>  vazy2["PolyProtein"]*

* *foreach subseq -> find the nsp that fits best* \
    1. indices comparison (auto)

In [99]:
def filter(tab:list, **kwargs) -> list:
    res:list = []
    for item in tab:
        params = [item[key] == value for key, value in kwargs.items()]
        #print(params)
        if all(params):
            res.append(item)
            #print(item)
            pass
        elif any(params):
            #print(item)
            pass
    return res

In [100]:
def indices_comparison(hits:list, **positions):
    result:list = []
    for hit in hits:
        bools = [hit[key] in interval(value) for key, value in positions.items()]
        #bools = [positions['start'] >= hit['start']-1 , positions['end'] <= hit['end']+1]
        #print(bools)
        if all(bools):
            result.append(hit)
        elif all([positions['start'] >= hit['start']-1 , positions['end'] <= hit['end']+1]):
            result.append(hit)
    return result

In [101]:
list_ex = [
{"PP": 1, "protein": 79, "index": 1, "start": 1, "end": 673},
{"PP": 1, "protein": 80, "index": 2, "start": 674, "end": 2779},
{"PP": 1, "protein": 81, "index": 3, "start": 2780, "end": 3086},
{"PP": 1, "protein": 82, "index": 4, "start": 3087, "end": 3379},
{"PP": 1, "protein": 83, "index": 5, "start": 3380, "end": 3462},
{"PP": 1, "protein": 84, "index": 6, "start": 3463, "end": 3672},
{"PP": 1, "protein": 85, "index": 7, "start": 3673, "end": 3783},
{"PP": 1, "protein": 86, "index": 8, "start": 3784, "end": 3928},
{"PP": 1, "protein": 87, "index": 9, "start": 3929, "end": 4868},
{"PP": 1, "protein": 88, "index": 10, "start": 4869, "end": 5468},
{"PP": 1, "protein": 89, "index": 11, "start": 5469, "end": 5989},
{"PP": 1, "protein": 90, "index": 12, "start": 5990, "end": 6327},
{"PP": 1, "protein": 91, "index": 13, "start": 6328, "end": 6629},]
print(indices_comparison(list_ex, start=2298, end=2779))
print(indices_comparison(list_ex, start=3929, end=4868))

[{'PP': 1, 'protein': 80, 'index': 2, 'start': 674, 'end': 2779}]
[{'PP': 1, 'protein': 87, 'index': 9, 'start': 3929, 'end': 4868}]


In [102]:
def interval(val:int, t:int=1):
    return range(val-t,val+t)

In [103]:
trace_transfer:dict = {}
#trace_no_nsp:dict = {}
trace_mlt_hit:dict = {}
trace_no_transfer:dict = {}

for modId, SubSeqs in vazi2_mod_trace.items():
    #print("modulo :", modId, sep='\t')
    trace_transfer[modId] = []
    for sseq in SubSeqs:
        #print("subseq :", sseq)
        items_filtered:list = filter(vazy2['PolyProtein'], PP=sseq[1])
        # filter management
        #
        #if len(items_filtered) < 1:
        #    if not modId in trace_no_nsp:
        #        trace_no_nsp[modId] = []
        #    sseq.append(False)
        #    trace_no_nsp[modId].append(sseq)    
        #    #print("no NSP for:", modId, sseq)
        #    break

        hits:list = indices_comparison(items_filtered, start=sseq[3], end=sseq[4])
        #print(*hits, sep='\n')

        # hit management
        if len(hits) > 1:
            """if not modId in trace_mlt_hit:
                trace_mlt_hit[modId] = []
            sseq.append(False)
            trace_mlt_hit[modId].append(sseq)
            #print("multiple hits", modId, sseq)"""
            break
        elif len(hits) < 1:
            #print("no hit:", modId, sseq)
            if not modId in trace_no_transfer:
                trace_no_transfer[modId] = []
            trace_no_transfer[modId].append(sseq)
            break
        ##
        
        trace_transfer[modId].append((sseq[0], hits[0]["protein"], sseq[2], hits[0]["start"], hits[0]["end"]))
        
            

        #break
        #endfor

    #break 
    #endfor



In [104]:
with open("./trace_transfer.json", 'w') as out :
    out.write(json.dumps(trace_transfer))
    
#with open("./trace_no_nsp.json", 'w') as out :
#    out.write(json.dumps(trace_no_nsp))
    
#with open("./trace_mlt_hit.json", 'w') as out :
#    out.write(json.dumps(trace_mlt_hit))

with open("./trace_no_transfer.json", 'w') as out :
    out.write(json.dumps(trace_no_transfer))


### alignment profile .vs. PP

In [48]:
def profile_alignment(modId, sseq):
    # profile
    fasta_path:str = "./aln/"+modId+'.aln.fasta'
    # input protein
    #with open("./sequence_input.fasta", 'w') as inFile:
    for protein in vazy2['Protein']:
        if protein['id'] == sseq[1]:
            text:str = ">"+protein['header']+"\n"+protein['sequence']
            #inFile.write('>'+protein['header']+'\n'+protein['sequence'])
            #command:str = f"muscle -in {filepath} -out {out_filepath} -quiet"
            return text



In [50]:
print(profile_alignment("F188", [23, 2, 6, 3784, 3928]))

>ORF1a_polyprotein__(@474)
MASSLKQGVSPKPRDVILVSKDIPEQLCDALFFYTSHNPKDYADAFAVRQKFDRSLQTGKQFKFETVCGLFLLKGVDKITPGVPAKVLKATSKLADLEDIFGVSPLARKYRELLKTACQWSLTVEALDVRAQTLDEIFDPTEILWLQVAAKIHVSSMAMRRLVGEVTAKVMDALGSNLSALFQIVKQQIARIFQKALAIFENVNELPQRIAALKMAFAKCARSITVVVVERTLVVKEFAGTCLASINGAVAKFFEELPNGFMGSKIFTTLAFFKEAAVRVVENIPNAPRGTKGFEVVGNAKGTQVVVRGMRNDLTLLDQKADIPVEPEGWSAILDGHLCYVFRSGDRFYAAPLSGNFALSDVHCCERVVCLSDGVTPEINDGLILAAIYSSFSVSELVTALKKGEPFKFLGHKFVYAKDAAVSFTLAKAATIADVLRLFQSARVIAEDVWSSFTEKSFEFWKLAYGKVRNLEEFVKTYVCKAQMSIVILAAVLGEDIWHLVSQVIYKLGVLFTKVVDFCDKHWKGFCVQLKRAKLIVTETFCVLKGVAQHCFQLLLDAIHSLYKSFKKCALGRIHGDLLFWKGGVHKIVQDGDEIWFDAIDSVDVEDLGVVQEKSIDFEVCDDVTLPENQPGHMVQIEDDGKNYMFFRFKKDENIYYTPMSQLGAINVVCKAGGKTVTFGETTVQEIPPPDVVPIKVSIECCGEPWNTIFKKAYKEPIEVDTDLTVEQLLSVIYEKMCDDLKLFPEAPEPPPFENVALVDKNGKDLDCIKSCHLIYRDYESDDDIEEEDAEECDTDSGEAEECDTNSECEEEDEDTKVLALIQDPASIKYPLPLDEDYSVYNGCIVHKDALDVVNLPSGEETFVVNNCFEGAVKPLPQKVVDVLGDWGEAVDAQEQLCQQEPLQHTFEEPVENSTGSSKTMTEQVVVEDQELPVVEQDQDVVVYTPTDLEVAKETAEEVDEFILIFAVPKEEV

# Comparator

*impossible cuz not same identification system*

In [98]:
import json, os 

vazi1_mod_trace:dict = {}
vazi2_mod_trace:dict = {}

with open("./vazi1_mod_trace.json") as handle :
    vazi1_mod_trace = json.load(handle)
with open("./vazi2_mod_trace.json") as handle :
    vazi2_mod_trace = json.load(handle)

# 